In [10]:
import casatools
import casatasks
import numpy as np
import glob, os
from astropy.coordinates import SkyCoord
from astropy import units as u

In [11]:
def find_central_time_range(ms_file, fraction=0.3):
    """
    Find the central time range of an MS (corrected version)
    
    Parameters
    ----------
    ms_file : str
        Path to measurement set
    fraction : float
        Fraction of total time to use (0.3 = central 30%)
    """
    
    # Use table tool to get time information directly
    tb = casatools.table()
    
    try:
        tb.open(ms_file)
        
        # Get all times from the main table
        times = tb.getcol('TIME')
        
        # Get unique times and sort them
        unique_times = np.unique(times)
        
        tb.close()
        
        # Calculate time range
        start_time = np.min(unique_times)
        end_time = np.max(unique_times)
        total_duration = end_time - start_time
        
        # Calculate central time range
        central_duration = total_duration * fraction
        central_start = start_time + (total_duration - central_duration) / 2
        central_end = central_start + central_duration
        
        # Convert to CASA time format
        qa = casatools.quanta()
        start_time_str = qa.time(qa.quantity(start_time, 's'), form='ymd')[0]
        end_time_str = qa.time(qa.quantity(end_time, 's'), form='ymd')[0]
        central_start_str = qa.time(qa.quantity(central_start, 's'), form='ymd')[0]
        central_end_str = qa.time(qa.quantity(central_end, 's'), form='ymd')[0]
        
        print(f"MS: {ms_file}")
        print(f"Full time range: {start_time_str} to {end_time_str}")
        print(f"Total duration: {total_duration/3600:.2f} hours")
        print(f"Central {fraction*100:.0f}% time range: {central_start_str} to {central_end_str}")
        print(f"Central duration: {central_duration/3600:.2f} hours")
        print(f"Central timerange string: '{central_start_str}~{central_end_str}'")
        
        return f"{central_start_str}~{central_end_str}"
        
    except Exception as e:
        print(f"Error reading times from {ms_file}: {e}")
        
        # Fallback: try with msmetadata
        try:
            msmd = casatools.msmetadata()
            msmd.open(ms_file)
            
            # Get times using msmetadata
            time_range = msmd.timerangeforobs()
            msmd.close()
            
            # Extract start and end times
            start_time = time_range['begin']['m0']['value']
            end_time = time_range['end']['m0']['value']
            total_duration = end_time - start_time
            
            # Calculate central range
            central_duration = total_duration * fraction
            central_start = start_time + (total_duration - central_duration) / 2
            central_end = central_start + central_duration
            
            # Convert to time strings
            qa = casatools.quanta()
            central_start_str = qa.time(qa.quantity(central_start, 's'), form='ymd')[0]
            central_end_str = qa.time(qa.quantity(central_end, 's'), form='ymd')[0]
            
            print(f"Using fallback method for {ms_file}")
            print(f"Central timerange: {central_start_str}~{central_end_str}")
            
            return f"{central_start_str}~{central_end_str}"
            
        except Exception as e2:
            print(f"Fallback method also failed: {e2}")
            return None

In [12]:
# Test the corrected function
central_timerange = find_central_time_range('/data/jfaber/dsa110-contimg/J1459_2025-05-29/msfiles/base/J1459_250529_60min_centered.ms', fraction=0.25)

MS: /data/jfaber/dsa110-contimg/J1459_2025-05-29/msfiles/base/J1459_250529_60min_centered.ms
Full time range: 2025/05/29/05:52:32 to 2025/05/29/07:00:48
Total duration: 1.14 hours
Central 25% time range: 2025/05/29/06:18:08 to 2025/05/29/06:35:12
Central duration: 0.28 hours
Central timerange string: '2025/05/29/06:18:08~2025/05/29/06:35:12'


In [13]:
def calibration(ms_file, cal_field_id, refant='pad103'):
    """
    More granular approach with splits at each step
    """
    
    base_name = ms_file.replace('.ms', '')
    
    print("=== STEP-BY-STEP CALIBRATION WITH SPLITS ===")
    
    # Step 1: Create a clean copy for calibration work
    print("\n1. Creating working copy...")
    working_ms = f"{base_name}_working.ms"
    
    if os.path.exists(working_ms):
        os.system(f'rm -rf {working_ms}')
    
    casatasks.split(
        vis=ms_file,
        outputvis=working_ms,
        datacolumn='data',
        keepflags=True  # Keep original flags
    )
    
    # Step 2: Initial flagging
    print("\n2. Flagging bad data...")
    casatasks.flagdata(vis=working_ms, mode='shadow', tolerance=0.0)
    casatasks.flagdata(vis=working_ms, mode='clip', clipzeros=True)
    
    # Step 3: Extract calibrator for bandpass
    print("\n3. Extracting calibrator for bandpass...")
    central_timerange = find_central_time_range(working_ms, fraction=0.3)
    
    cal_for_bp_ms = f"{base_name}_cal_for_bandpass.ms"
    if os.path.exists(cal_for_bp_ms):
        os.system(f'rm -rf {cal_for_bp_ms}')
    
    casatasks.split(
        vis=working_ms,
        outputvis=cal_for_bp_ms,
        field=str(cal_field_id),
        timerange=central_timerange,
        datacolumn='data'
    )
    
    # Step 4: Bandpass calibration
    print("\n4. Bandpass calibration...")
    bp_table = f"{base_name}_bandpass.bcal"
    
    casatasks.bandpass(
        vis=cal_for_bp_ms,
        caltable=bp_table,
        refant=refant,
        bandtype='B',
        solint='inf',
        combine='scan',
        minsnr=3.0,
        fillgaps=80
    )
    
    # Step 5: Apply bandpass and create new MS
    print("\n5. Applying bandpass...")
    bp_applied_ms = f"{base_name}_bp_applied.ms"
    
    if os.path.exists(bp_applied_ms):
        os.system(f'rm -rf {bp_applied_ms}')
    
    casatasks.applycal(
        vis=working_ms,
        gaintable=[bp_table],
        calwt=False
    )
    
    casatasks.split(
        vis=working_ms,
        outputvis=bp_applied_ms,
        datacolumn='corrected'
    )
    
    # Step 6: Gain calibration on bandpass-corrected data
    print("\n6. Gain calibration...")
    
    # Extract just calibrator from bandpass-corrected data
    cal_bp_ms = f"{base_name}_cal_bp_corrected.ms"
    if os.path.exists(cal_bp_ms):
        os.system(f'rm -rf {cal_bp_ms}')
    
    casatasks.split(
        vis=bp_applied_ms,
        outputvis=cal_bp_ms,
        field=str(cal_field_id)
    )
    
    # Phase calibration
    phase_table = f"{base_name}_phase.gcal"
    casatasks.gaincal(
        vis=cal_bp_ms,
        caltable=phase_table,
        gaintype='G',
        solint='60s',
        refant=refant,
        calmode='p',
        minsnr=3.0
    )
    
    # Amplitude calibration
    amp_table = f"{base_name}_amp.gcal"
    casatasks.gaincal(
        vis=cal_bp_ms,
        caltable=amp_table,
        gaintype='G',
        solint='60s',
        refant=refant,
        calmode='ap',
        gaintable=[phase_table],
        minsnr=3.0
    )
    
    # Step 7: Apply all calibrations and create final MS
    print("\n7. Creating final calibrated MS...")
    final_ms = f"{base_name}_final_calibrated.ms"
    
    if os.path.exists(final_ms):
        os.system(f'rm -rf {final_ms}')
    
    casatasks.applycal(
        vis=bp_applied_ms,
        gaintable=[bp_table, amp_table],
        calwt=False
    )
    
    casatasks.split(
        vis=bp_applied_ms,
        outputvis=final_ms,
        datacolumn='corrected'
    )
    
    print("\n=== FILES CREATED ===")
    print(f"Working copy: {working_ms}")
    print(f"Bandpass-corrected: {bp_applied_ms}")
    print(f"Final calibrated: {final_ms}")
    print(f"Calibration tables: {bp_table}, {phase_table}, {amp_table}")
    
    return {
        'working_ms': working_ms,
        'bp_applied_ms': bp_applied_ms,
        'final_ms': final_ms,
        'tables': [bp_table, phase_table, amp_table]
    }

In [14]:
# Run with correct field
ms_dict = calibration('/data/jfaber/dsa110-contimg/J1459_2025-05-29/msfiles/base/mscal/J1459_250529_60min_centered.ms', cal_field_id=117, refant='pad103')

=== STEP-BY-STEP CALIBRATION WITH SPLITS ===

1. Creating working copy...

2. Flagging bad data...


2025-06-03 02:20:00	SEVERE	MeasTable::dUTC(Double) (file /source/casa6/casatools/casacore/measures/Measures/MeasTable.cc, line 4290)	Leap second table TAI_UTC seems out-of-date.
2025-06-03 02:20:00	SEVERE	MeasTable::dUTC(Double) (file /source/casa6/casatools/casacore/measures/Measures/MeasTable.cc, line 4290)+	Until the table is updated (see the CASA documentation or your system admin),
2025-06-03 02:20:00	SEVERE	MeasTable::dUTC(Double) (file /source/casa6/casatools/casacore/measures/Measures/MeasTable.cc, line 4290)+	times and coordinates derived from UTC could be wrong by 1s or more.



3. Extracting calibrator for bandpass...
MS: /data/jfaber/dsa110-contimg/J1459_2025-05-29/msfiles/base/mscal/J1459_250529_60min_centered_working.ms
Full time range: 2025/05/29/05:52:32 to 2025/05/29/07:00:48
Total duration: 1.14 hours
Central 30% time range: 2025/05/29/06:16:26 to 2025/05/29/06:36:55
Central duration: 0.34 hours
Central timerange string: '2025/05/29/06:16:26~2025/05/29/06:36:55'

4. Bandpass calibration...


2025-06-03 02:22:53	WARN	MSDerivedValues::parAngle	unhandled mount type
2025-06-03 02:22:53	WARN	MSDerivedValues::parAngle	unhandled mount type
2025-06-03 02:22:53	WARN	MSDerivedValues::parAngle	unhandled mount type
2025-06-03 02:22:53	WARN	MSDerivedValues::parAngle	unhandled mount type
2025-06-03 02:22:53	WARN	MSDerivedValues::parAngle	unhandled mount type
2025-06-03 02:22:53	WARN	MSDerivedValues::parAngle	unhandled mount type
2025-06-03 02:22:53	WARN	MSDerivedValues::parAngle	unhandled mount type
2025-06-03 02:22:53	WARN	MSDerivedValues::parAngle	unhandled mount type
2025-06-03 02:22:53	WARN	MSDerivedValues::parAngle	unhandled mount type
2025-06-03 02:22:53	WARN	MSDerivedValues::parAngle	unhandled mount type
2025-06-03 02:22:53	WARN	MSDerivedValues::parAngle	unhandled mount type
2025-06-03 02:22:53	WARN	MSDerivedValues::parAngle	unhandled mount type
2025-06-03 02:22:53	WARN	MSDerivedValues::parAngle	unhandled mount type
2025-06-03 02:22:53	WARN	MSDerivedValues::parAngle	unhandled mou

30 of 188 solutions flagged due to SNR < 3 in spw=0 (chan=767) at 2025/05/29/06:24:17.7
31 of 188 solutions flagged due to SNR < 3 in spw=0 (chan=766) at 2025/05/29/06:24:17.7
5 of 188 solutions flagged due to SNR < 3 in spw=0 (chan=765) at 2025/05/29/06:24:17.7
2 of 188 solutions flagged due to SNR < 3 in spw=0 (chan=764) at 2025/05/29/06:24:17.7
1 of 188 solutions flagged due to SNR < 3 in spw=0 (chan=763) at 2025/05/29/06:24:17.7
1 of 188 solutions flagged due to SNR < 3 in spw=0 (chan=762) at 2025/05/29/06:24:17.7
2 of 188 solutions flagged due to SNR < 3 in spw=0 (chan=761) at 2025/05/29/06:24:17.7
1 of 188 solutions flagged due to SNR < 3 in spw=0 (chan=760) at 2025/05/29/06:24:17.7
1 of 188 solutions flagged due to SNR < 3 in spw=0 (chan=759) at 2025/05/29/06:24:17.7
1 of 188 solutions flagged due to SNR < 3 in spw=0 (chan=758) at 2025/05/29/06:24:17.7
1 of 188 solutions flagged due to SNR < 3 in spw=0 (chan=757) at 2025/05/29/06:24:17.7
2 of 188 solutions flagged due to SNR < 3

2025-06-03 02:29:47	WARN	MSTransformManager::checkDataColumnsToFill	CORRECTED_DATA column requested but not available in input MS 
2025-06-03 02:29:47	SEVERE	split::::casa	Task split raised an exception of class RuntimeError with the following message: Desired column (CORRECTED_DATA) not found in the input MS (/data/jfaber/dsa110-contimg/J1459_2025-05-29/msfiles/base/mscal/J1459_250529_60min_centered_bp_applied.ms).


RuntimeError: Desired column (CORRECTED_DATA) not found in the input MS (/data/jfaber/dsa110-contimg/J1459_2025-05-29/msfiles/base/mscal/J1459_250529_60min_centered_bp_applied.ms).